# Pixel -> Stage calibration for overviews

In [1]:
import os
import logging
import json

from nis_util import do_large_image_scan, set_optical_configuration, get_position

logging.basicConfig(format='%(asctime)s - %(levelname)s in %(funcName)s: %(message)s', level=logging.DEBUG)
logger = logging.getLogger(__name__)

## set paths

In [46]:
path_to_nis = 'C:\\Program Files\\NIS-Elements\\nis_ar.exe'
save_base_path = 'C:\\Users\\Nikon\\Documents\\David\\overview_calibrations'
calibration_name = 'right_260418.json'

if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
    
if os.path.exists(os.path.join(save_base_path, calibration_name)):
    logger.warning('output file already exists, will be overwritten')

## do a dummy overview scan

please enter the coordinates of the field you wish to calibrate as (left, right, top, bottom) and the optical configuration you are using for overviews.

Running the cell will do a dummy overview and leave the resulting image open in NIS.

**! The z-position will be recorded to the calibration file, so please focus on a slide !**

In [47]:
calibration_fov = (-26500, -50000, -21053, 18177)
calibration_oc = 'DIA4x'
left, right, top, bottom = calibration_fov

set_optical_configuration(path_to_nis, calibration_oc)
do_large_image_scan(path_to_nis, '', left, right, top, bottom, close=False)
pos = get_position(path_to_nis)

## corresponding points

Now, please collect 3 pairs of pixel (x,y)-coordinates and stage (um) coordinates.
To do so, hover over the image, the pixel coordinates will be displayed on the bottom of the image window.
If you right-click, the corresponding stage coordinates will be shown under ```Move this point to center```.

**if you use binning, please multiply the *pixel coordinates* with the binning factor** 

In [49]:
coords1px = (1739*2, 1306*2)
coords1st = (-30451, -18605)

coords2px = (7179*2, 5005*2)
coords2st = (-48144, -6822)

coords3px = (1168*2, 7243*2)
coords3st = (-28791, 596)

## save the calibration

execute the following to save the calibration as a JSON-file.

In [50]:
res = {}
res['bbox'] = calibration_fov
res['coords_px'] = [coords1px, coords2px, coords3px]
res['coords_st'] = [coords1st, coords2st, coords3st]
res['zpos'] = pos[2]

with open(os.path.join(save_base_path, calibration_name), 'w') as fd:
    json.dump(res, fd, indent=1)
    print('saved calibration: \n\n{}'.format(json.dumps(res, indent=1)))

saved calibration: 

{
 "bbox": [
  -26500,
  -50000,
  -21053,
  18177
 ],
 "coords_px": [
  [
   3478,
   2612
  ],
  [
   14358,
   10010
  ],
  [
   2336,
   14486
  ]
 ],
 "coords_st": [
  [
   -30451,
   -18605
  ],
  [
   -48144,
   -6822
  ],
  [
   -28791,
   596
  ]
 ]
}


## test the transformation

below, you can test the transformation by entering new pixel coordinates and checking if the output corresponds to the stage coordinates shown in NIS. 

In [51]:
import numpy as np
from skimage.transform import AffineTransform

test_coords = (5954*2, 10218*2)

coords_px = np.array([coords1px, coords2px, coords3px], dtype=np.float)
coords_st = np.array([coords1st, coords2st, coords3st], dtype=np.float)

at = AffineTransform()
at.estimate(coords_px, coords_st)
at(np.array(test_coords)).ravel()

array([-44348.23315942,  10059.97102801])

In [52]:
at.params

array([[ -1.61559998e+00,  -1.55815376e-02,  -2.47912443e+04],
       [ -1.55310427e-02,   1.61556877e+00,  -2.27708487e+04],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])